In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir('drive/Colab/sentiment_analysis')

In [ ]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling2D , Add , Merge
from keras.layers import Conv2D, MaxPooling2D, Embedding , Reshape , Flatten
from keras.models import Model
from keras.models import load_model
from keras import optimizers
import csv
import os
GLOVE_DIR = "drive/Colab/sentiment_analysis/glove/"
MAX_SEQUENCE_LENGTH = 250
MAX_NUM_WORDS = 400000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.3

In [ ]:
embeddings_index = {}
with open("glove.6B.100d.txt") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
texts = []  # list of text samples
labels_index = {"positive" : 2 , "neutral" : 1 , "negative" : 0}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

full_dataset = []
with open("preprocessed_train_new.csv", "r" , encoding = 'utf8') as f:
	reader = csv.reader(f)
	for row in reader:
		full_dataset.append(row)
full_dataset = full_dataset[1:]
print('Found %s texts.' % len(full_dataset))

texts = [i[2] for i in full_dataset]
labels = [int(i[1]) for i in full_dataset]



Found 400001 texts.


In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))

Found 216118 unique tokens.


In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
eval_data = data[len(data) - 40000 :]
eval_labels = labels[len(labels) - 40000:]

data = data[0:len(data) - 40000]
labels = labels[0:len(labels) - 40000]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = Embedding(num_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
print('Training model cnn2d')
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Reshape((1, MAX_SEQUENCE_LENGTH, 100))(embedded_sequences)
print(x.shape)
x = Conv2D(15, 3, activation='tanh' , padding = 'same')(x)
x = MaxPooling2D(pool_size = (3,3),dim_ordering="th")(x)
# x = Conv2D(3, 3, activation='relu' , padding = 'same')(x)
# x = MaxPooling2D(pool_size = (2,2),dim_ordering="th")(x)
x = Flatten()(x)
preds = Dense(len(labels_index), activation='softmax')(x)
model = Model(sequence_input, preds)
adam = optimizers.Adam(lr = 0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=64,
          epochs=15,
          validation_data=(x_val, y_val))


In [ ]:
model.save("cnn2d_sigmoid40.hdf5")

In [ ]:
loss , acc = model.evaluate(eval_data , eval_labels , batch_size = 32)
print(loss , acc)

40000/40000 [==============================] - 4s 108us/step
0.20881551322788 0.9177333537101746


In [ ]:
model_tanh = load_model("cnn2d_tanh.model")

In [ ]:
model_tanh.save("cnn2d_tanh.hdf5")